<a href="https://colab.research.google.com/github/PPjaisri/Senior-project/blob/Lion/News_Thai_Text_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# แปลงเป็น Bag of words ข่าวจริง - จริง

In [ ]:
!pip install pythainlp
!pip install stop_words
import pythainlp
import numpy
from pythainlp import word_tokenize
from pythainlp import corpus
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from string import punctuation

In [ ]:
import nltk
nltk.download('words')
th_stop = thai_stopwords()
p_stemmer = PorterStemmer()

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
def split_word(text):
            
    
    tokens = word_tokenize(text,engine='newmm')
    
    # Remove stop words ภาษาไทย และอักขระพิเศษ
    tokens = [i for i in tokens if not i in th_stop and not i in punctuation] 
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens

In [ ]:
def find_cosine_similarity(news_array, first_part_index, second_part_index):
    sum = 0 #upper part of cosine similarity
    discount = 0 #how many 0, 0 pair word count in both first_part and second part
    total_words_number = news_array.shape[1]
    for index in range(total_words_number):
        if (news_array[first_part_index, index] == 0) and (news_array[second_part_index, index] ==0):
            discount += 1
            pass
        sum += news_array[first_part_index, index]*news_array[second_part_index, index]
    print('This is sum (upperpart) :',sum)
    total_words_after_discount = total_words_number-discount
    lowerpart = total_words_after_discount * total_words_after_discount
    print('This is lowerpart :',lowerpart)
    return (sum/lowerpart)

# ข่าวที่ 1

In [ ]:
clean_text = ["น้ำจะท่วมกรุงเทพฯ ในเดือนตุลา 64","โดยสรุปก็คึอ ถ้าดูจากตัวเลขปริมาณการไหลของน้ำ การระบายน้ำ ปริมาณน้ำในเขื่อนใหญ่ที่ยังน้อยมาก และยังมีพื้นที่รับน้ำได้อีกมาก รวมทั้งพื้นที่รับน้ำที่ท่วมอยู่ในเวลานี้ด้วย ก็จะเห็นว่า กรุงเทพมหานคร และนนทบุรี ยังคงไม่มีแนวโน้มจะถูกน้ำท่วม (ที่ไม่ใช่น้ำท่วมขังเวลาฝนตก)", "น้ำท่วมจะไปไหน ข่าวลือเริ่มแรง กรุงเทพฯ-นนทบุรี จะจมน้ำ?", "แหล่งข่าวผู้เชี่ยวชาญด้านทรัพยากรน้ำระบุ สถานการณ์น้ำในปัจจุบัน จากข้อมูลพบว่ายังคงแตกต่างจากปี 2554 หลังเกิดความกังวล ส่งต่อข้อมูลและเปรียบเทียบว่า น้ำจะท่วมกรุงเทพมหานครและนนทบุรีแบบมหาอุทกภัย"]

**Step 2: Tokenize**

In [ ]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['น้ำ', 'โชก', 'กทม.', 'เดือน', 'ต.ค.']
tokenized text:
 ['โดยสรุป', 'คึอ', 'ดู', 'เลข', 'จำนวน', 'ไหล', 'น้ำ', 'ระบาย', 'น้ำ', 'จำนวน', 'น้ำ', 'เขื่อน', 'ยังมี', 'พื้นที่รับน้ำ', 'พื้นที่รับน้ำ', 'โชก', 'ในเวลานี้', 'กทม.', 'นนทบุรี', 'มีแนวโน้ม', 'ท่วม', 'ท่วม', 'ขัง', 'กาล', 'ฝนตก']
tokenized text:
 ['ท่วม', 'ข่าวลือ', 'แรง', 'กทม.', 'นนทบุรี', 'จมน้ำ']
tokenized text:
 ['แหล่งข่าว', 'ผู้เชี่ยวชาญ', 'ทรัพยากร', 'น้ำ', 'ระบุ', 'สถานการณ์', 'น้ำ', 'ข้อมูล', 'ต่าง', 'ปี', 'ความกังวล', 'ส่งต่อ', 'ข้อมูล', 'เปรียบเทียบ', 'น้ำ', 'โชก', 'กทม.', 'นนทบุรี', 'มหา', 'อุทกภัย']


In [ ]:
tokens_list = [split_word(txt) for txt in clean_text]

In [ ]:
tokens_list

[['น้ำ', 'โชก', 'กทม.', 'เดือน', 'ต.ค.'],
 ['โดยสรุป',
  'คึอ',
  'ดู',
  'เลข',
  'จำนวน',
  'ไหล',
  'น้ำ',
  'ระบาย',
  'น้ำ',
  'จำนวน',
  'น้ำ',
  'เขื่อน',
  'ยังมี',
  'พื้นที่รับน้ำ',
  'พื้นที่รับน้ำ',
  'โชก',
  'ในเวลานี้',
  'กทม.',
  'นนทบุรี',
  'มีแนวโน้ม',
  'ท่วม',
  'ท่วม',
  'ขัง',
  'กาล',
  'ฝนตก'],
 ['ท่วม', 'ข่าวลือ', 'แรง', 'กทม.', 'นนทบุรี', 'จมน้ำ'],
 ['แหล่งข่าว',
  'ผู้เชี่ยวชาญ',
  'ทรัพยากร',
  'น้ำ',
  'ระบุ',
  'สถานการณ์',
  'น้ำ',
  'ข้อมูล',
  'ต่าง',
  'ปี',
  'ความกังวล',
  'ส่งต่อ',
  'ข้อมูล',
  'เปรียบเทียบ',
  'น้ำ',
  'โชก',
  'กทม.',
  'นนทบุรี',
  'มหา',
  'อุทกภัย']]

**Step 3 + 4a: Bag of words + count word**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x7f37ba5ec320>, binary=False,
                decode_error='strict', dtype=<class 'numpy.int64'>,
                encoding='utf-8', input='content', lowercase=True, max_df=1.0,
                max_features=None, min_df=1, ngram_range=(1, 1),
                preprocessor=None, stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)


In [ ]:
cvec.vocabulary_

{'กทม.': 0,
 'กาล': 1,
 'ขัง': 2,
 'ข่าวลือ': 3,
 'ข้อมูล': 4,
 'ความกังวล': 5,
 'คึอ': 6,
 'จมน้ำ': 7,
 'จำนวน': 8,
 'ดู': 9,
 'ต.ค.': 10,
 'ต่าง': 11,
 'ทรัพยากร': 12,
 'ท่วม': 13,
 'นนทบุรี': 14,
 'น้ำ': 15,
 'ปี': 16,
 'ผู้เชี่ยวชาญ': 17,
 'ฝนตก': 18,
 'พื้นที่รับน้ำ': 19,
 'มหา': 20,
 'มีแนวโน้ม': 21,
 'ยังมี': 22,
 'ระบาย': 23,
 'ระบุ': 24,
 'สถานการณ์': 25,
 'ส่งต่อ': 26,
 'อุทกภัย': 27,
 'เขื่อน': 28,
 'เดือน': 29,
 'เปรียบเทียบ': 30,
 'เลข': 31,
 'แรง': 32,
 'แหล่งข่าว': 33,
 'โชก': 34,
 'โดยสรุป': 35,
 'ในเวลานี้': 36,
 'ไหล': 37}

In [ ]:
c_feat[:,:].todense()

matrix([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 0, 2, 1, 0, 0, 0, 2, 1, 3, 0, 0, 1, 2, 0,
         1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 3, 1, 1, 0, 0, 1,
         0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]])

In [ ]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
  0 0]
 [1 1 1 0 0 0 1 0 2 1 0 0 0 2 1 3 0 0 1 2 0 1 1 1 0 0 0 0 1 0 0 1 0 0 1 1
  1 1]
 [1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  0 0]
 [1 0 0 0 2 1 0 0 0 0 0 1 1 0 1 3 1 1 0 0 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0
  0 0]]


In [ ]:
first_news.shape[1]

38

In [ ]:
#แปลงเป็น array ทััง 3 ข่าวเพื่อหา cosine similarity
cofact_checker_array = numpy.array(cofact_checker) 
title_news_array = numpy.array(title_news) 
first_paragraph_news_array = numpy.array(first_paragraph_news)

#หา similarity เทียบระหว่างข่าวที่ต้องการตรวจสอบบน cofact กับหัวข้อข่าวที่เชื่อถือได้
print(cofact_checker_array[0][2])
print(first_paragraph_news_array[0][4])

1
2


In [ ]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 5
This is lowerpart : 484


0.010330578512396695

In [ ]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 1
This is lowerpart : 100


0.01

In [ ]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 5
This is lowerpart : 361


0.013850415512465374

# -------------- ข่าวที่ 2 ----------------------------

In [ ]:
clean_text = ["ขณะนี้ คิวในการยื่นขอ VISA ประเทศสหรัฐอเมริกา คิวอยู่ที่เดือนสิงหาคม พ.ศ. 2565 แล้ว จริงหรือไม่","ระยะเวลารอคิวการยื่นวีซ่าสามารถเช็กได้ที่เว็บไซต์https://travel.state.gov/content/travel/⋯sa-information-resources/wait-times.htmlจากการเช็กล่าสุดพบว่าระยะเวลาการรอวีซ่าคือVisitor Visa 35 Calendar DaysStudent/Exchange Visitor Visas 2 Calendar DaysAll Other Nonimmigrant Visas 2 Calendar Daysไม่ถึงปี 65 แต่อย่างใด"]

**Step 2: Tokenize**

In [ ]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
# print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
# print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['คิว', 'ยื่น', 'VISA', 'รัฐ', 'สหรัฐอเมริกา', 'คิว', 'เดือน', 'ส.ค.', 'พ.ศ.']
tokenized text:
 ['ระยะเวลา', 'รอ', 'คิว', 'ยื่น', 'วีซ่า', 'เช็ก', 'เว็บ', 'https', '://', 'travel', 'state', 'gov', 'content', 'travel', '/⋯', 'sa-information-resources', 'wait-times', 'html', 'เช็ก', 'ระยะเวลา', 'รอ', 'วีซ่า', 'Visitor', 'Visa', 'Calendar', 'DaysStudent', 'Exchange', 'Visitor', 'Visas', 'Calendar', 'DaysAll', 'Other', 'Nonimmigrant', 'Visas', 'Calendar', 'Days', 'ปี']


In [ ]:
tokens_list = [split_word(txt) for txt in clean_text]

In [ ]:
tokens_list

[['คิว',
  'ยื่น',
  'VISA',
  'รัฐ',
  'สหรัฐอเมริกา',
  'คิว',
  'เดือน',
  'ส.ค.',
  'พ.ศ.'],
 ['ระยะเวลา',
  'รอ',
  'คิว',
  'ยื่น',
  'วีซ่า',
  'เช็ก',
  'เว็บ',
  'https',
  '://',
  'travel',
  'state',
  'gov',
  'content',
  'travel',
  '/⋯',
  'sa-information-resources',
  'wait-times',
  'html',
  'เช็ก',
  'ระยะเวลา',
  'รอ',
  'วีซ่า',
  'Visitor',
  'Visa',
  'Calendar',
  'DaysStudent',
  'Exchange',
  'Visitor',
  'Visas',
  'Calendar',
  'DaysAll',
  'Other',
  'Nonimmigrant',
  'Visas',
  'Calendar',
  'Days',
  'ปี']]

**Step 3 + 4a: Bag of words + count word**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [ ]:
cvec.vocabulary_

{'/⋯': 0,
 '://': 1,
 'Calendar': 2,
 'Days': 3,
 'DaysAll': 4,
 'DaysStudent': 5,
 'Exchange': 6,
 'Nonimmigrant': 7,
 'Other': 8,
 'VISA': 9,
 'Visa': 10,
 'Visas': 11,
 'Visitor': 12,
 'content': 13,
 'gov': 14,
 'html': 15,
 'https': 16,
 'sa-information-resources': 17,
 'state': 18,
 'travel': 19,
 'wait-times': 20,
 'คิว': 21,
 'ปี': 22,
 'พ.ศ.': 23,
 'ยื่น': 24,
 'รอ': 25,
 'ระยะเวลา': 26,
 'รัฐ': 27,
 'วีซ่า': 28,
 'ส.ค.': 29,
 'สหรัฐอเมริกา': 30,
 'เช็ก': 31,
 'เดือน': 32,
 'เว็บ': 33}

In [ ]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         2, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0],
        [1, 1, 3, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1,
         1, 1, 0, 1, 2, 2, 0, 2, 0, 0, 2, 0, 1]])

In [ ]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 2 0 1 1 0 0 1 0 1 1 0 1 0]
 [1 1 3 1 1 1 1 1 1 0 1 2 2 1 1 1 1 1 1 2 1 1 1 0 1 2 2 0 2 0 0 2 0 1]]


In [ ]:
first_news.shape[1]

34

In [ ]:
#แปลงเป็น array ทััง 3 ข่าวเพื่อหา cosine similarity
cofact_checker_array = numpy.array(cofact_checker) 
title_news_array = numpy.array(title_news) 
first_paragraph_news_array = numpy.array(first_paragraph_news)

#หา similarity เทียบระหว่างข่าวที่ต้องการตรวจสอบบน cofact กับหัวข้อข่าวที่เชื่อถือได้

In [ ]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 3
This is lowerpart : 1156


0.0025951557093425604

# ข่าวที่ 3

In [ ]:
clean_text = ["ประเทศไทย มีจังหวัดเพิ่มอีก 6 จังหวัด จริงหรือ","ผู้สื่อข่าวรายงานว่า กรมการปกครอง กระทรวงมหาดไทย เผยแพร่ข้อมูลว่าตามที่ปรากฏข่าวสารผ่านสื่อออนไลน์ว่า “ประกาศชาวไทย จะมี 83 จังหวัดแล้ว” นั้น กรมการปกครอง กระทรวงมหาดไทย ชี้แจงว่า ข้อมูลข่าวดังกล่าวไม่เป็นความจริงแต่ประการใด และเป็นการแชร์ข่าวที่เคยเป็นข่าวปลอม มาตั้งแต่ปี พ.ศ.2558 แล้ว","มท.ยืนยันประเทศไทยมีแค่ 76 จังหวัด","กรมการปกครอง ร่อนหนังสือชี้แจงข่าวปลอมในโซเชียล อ้างประกาศชาวไทยจะมี 83 จังหวัด ชี้เป็นข่าวปลอมตั้งแต่ปี 2558 ยืนยันไทยมี 76 จังหวัด"]

**Step 2: Tokenize**

In [ ]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['รัฐ', 'ไท', 'รัฐ', 'รัฐ']
tokenized text:
 ['สื่อ', 'รายงาน', 'กรมการปกครอง', 'กระทรวงมหาดไทย', 'โฆษณา', 'ข้อมูล', 'ข่าวสาร', 'สื่อ', 'ออนไลน์', '“', 'ประกาศ', 'คนไทย', 'รัฐ', '”', 'กรมการปกครอง', 'กระทรวงมหาดไทย', 'ชี้แจง', 'ข้อมูล', 'ข่าว', 'ความจริง', 'แต่ประการใด', 'แชร์', 'ข่าว', 'เป็นข่าว', 'ปลอม', 'ปี', 'พ.ศ.']
tokenized text:
 ['มท', 'รัฐ', 'ไท', 'รัฐ']
tokenized text:
 ['กรมการปกครอง', 'ร่อน', 'สมุด', 'ชี้แจง', 'ข่าว', 'ปลอม', 'โซเชียล', 'อ้าง', 'ประกาศ', 'คนไทย', 'รัฐ', 'ชี้', 'เป็นข่าว', 'ปลอม', 'ปี', 'ไท', 'รัฐ']


In [ ]:
tokens_list = [split_word(txt) for txt in clean_text]

In [ ]:
tokens_list

[['รัฐ', 'ไท', 'รัฐ', 'รัฐ'],
 ['สื่อ',
  'รายงาน',
  'กรมการปกครอง',
  'กระทรวงมหาดไทย',
  'โฆษณา',
  'ข้อมูล',
  'ข่าวสาร',
  'สื่อ',
  'ออนไลน์',
  '“',
  'ประกาศ',
  'คนไทย',
  'รัฐ',
  '”',
  'กรมการปกครอง',
  'กระทรวงมหาดไทย',
  'ชี้แจง',
  'ข้อมูล',
  'ข่าว',
  'ความจริง',
  'แต่ประการใด',
  'แชร์',
  'ข่าว',
  'เป็นข่าว',
  'ปลอม',
  'ปี',
  'พ.ศ.'],
 ['มท', 'รัฐ', 'ไท', 'รัฐ'],
 ['กรมการปกครอง',
  'ร่อน',
  'สมุด',
  'ชี้แจง',
  'ข่าว',
  'ปลอม',
  'โซเชียล',
  'อ้าง',
  'ประกาศ',
  'คนไทย',
  'รัฐ',
  'ชี้',
  'เป็นข่าว',
  'ปลอม',
  'ปี',
  'ไท',
  'รัฐ']]

**Step 3 + 4a: Bag of words + count word**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x7f37aefd7ef0>, binary=False,
                decode_error='strict', dtype=<class 'numpy.int64'>,
                encoding='utf-8', input='content', lowercase=True, max_df=1.0,
                max_features=None, min_df=1, ngram_range=(1, 1),
                preprocessor=None, stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)


In [ ]:
cvec.vocabulary_

{'กรมการปกครอง': 0,
 'กระทรวงมหาดไทย': 1,
 'ข่าว': 2,
 'ข่าวสาร': 3,
 'ข้อมูล': 4,
 'คนไทย': 5,
 'ความจริง': 6,
 'ชี้': 7,
 'ชี้แจง': 8,
 'ประกาศ': 9,
 'ปลอม': 10,
 'ปี': 11,
 'พ.ศ.': 12,
 'มท': 13,
 'รัฐ': 14,
 'รายงาน': 15,
 'ร่อน': 16,
 'สมุด': 17,
 'สื่อ': 18,
 'ออนไลน์': 19,
 'อ้าง': 20,
 'เป็นข่าว': 21,
 'แชร์': 22,
 'แต่ประการใด': 23,
 'โฆษณา': 24,
 'โซเชียล': 25,
 'ไท': 26,
 '“': 27,
 '”': 28}

In [ ]:
c_feat[:,:].todense()

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0],
        [2, 2, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 2, 1, 0,
         1, 1, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 2, 1, 0, 0, 2, 0, 1, 1, 0, 0, 1,
         1, 0, 0, 0, 1, 1, 0, 0]])

In [ ]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [2 2 2 1 2 1 1 0 1 1 1 1 1 0 1 1 0 0 2 1 0 1 1 1 1 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 1 0 0 1 0 1 1 1 2 1 0 0 2 0 1 1 0 0 1 1 0 0 0 1 1 0 0]]


In [ ]:
first_news.shape[1]

29

In [ ]:
#แปลงเป็น array ทััง 3 ข่าวเพื่อหา cosine similarity
cofact_checker_array = numpy.array(cofact_checker) 
title_news_array = numpy.array(title_news) 
first_paragraph_news_array = numpy.array(first_paragraph_news)

#หา similarity เทียบระหว่างข่าวที่ต้องการตรวจสอบบน cofact กับหัวข้อข่าวที่เชื่อถือได้
print(cofact_checker_array[0][2])
print(first_paragraph_news_array[0][4])

1
2


In [ ]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 3
This is lowerpart : 529


0.005671077504725898

In [ ]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 9


0.7777777777777778

In [ ]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 7
This is lowerpart : 225


0.03111111111111111

# ข่าวที่ 4

In [ ]:
clean_text = ["ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม", "นายอำเภอเมืองยะลา นำทีมลงพื้นที่ กำชับมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่มเคร่งครัด เพื่อลดความเสี่ยงแพร่เชื้อโควิด-19 ที่แพร่ระบาดอย่างรวดเร็ว รวมถึง คลัสเตอร์ใหญ่ในพื้นที่จังหวัดยะลา (โรงเรียน โรงงานยาง ยะลา เบตง) มียอดผู้ติดเชื้อเพิ่มสูงขึ้นอย่างต่อเนื่อง", "ยะลา คุมเข้มมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่ม ลดเสี่ยงโควิด-19","นายอำเภอเมืองยะลา นำทีมลงพื้นที่ กำชับมาตรการ เคอร์ฟิว - ร้านขายอาหาร เครื่องดื่ม ปิด 2 ทุ่มเคร่งครัด เพื่อลดความเสี่ยงแพร่เชื้อโควิด-19"]

**Step 2: Tokenize**

In [ ]:
print('tokenized text:\n',split_word(clean_text[0])) #ข้อความที่ต้องการตรวจสอบบน cofact
print('tokenized text:\n',split_word(clean_text[1])) #ข้อความสรุปใน cofact
print('tokenized text:\n',split_word(clean_text[2])) #หัวข่าวอ้างอิง
print('tokenized text:\n',split_word(clean_text[3])) #เนื้อหาข่าวอ้างอิง paragraph แรก

tokenized text:
 ['ยะลา', 'คุม', 'เข้ม', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่องดื่ม', 'ทุ่ม']
tokenized text:
 ['นอ.', 'เมือง', 'ยะลา', 'ทีม', 'พื้นที่', 'ย้ำ', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่องดื่ม', 'ทุ่ม', 'เคร่งครัด', 'ลด', 'ความเสี่ยง', 'แพร่เชื้อ', 'โควิด', 'แพร่ระบาด', 'คลัสเตอร์', 'พื้นที่', 'รัฐ', 'ยะลา', 'ร.ร.', 'ร.ง.', 'ยาง', 'ยะลา', 'เบตง', 'ยอด', 'ติดโรค', 'ต่อเนื่อง']
tokenized text:
 ['ยะลา', 'คุม', 'เข้ม', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่องดื่ม', 'ทุ่ม', 'ลด', 'เสี่ยง', 'โควิด']
tokenized text:
 ['นอ.', 'เมือง', 'ยะลา', 'ทีม', 'พื้นที่', 'ย้ำ', 'มาตรการ', 'เคอร์ฟิว', 'ร้าน', 'ขาย', 'อาหาร', 'เครื่องดื่ม', 'ทุ่ม', 'เคร่งครัด', 'ลด', 'ความเสี่ยง', 'แพร่เชื้อ', 'โควิด']


In [ ]:
tokens_list = [split_word(txt) for txt in clean_text]

In [ ]:
tokens_list

[['ยะลา',
  'คุม',
  'เข้ม',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่องดื่ม',
  'ทุ่ม'],
 ['นอ.',
  'เมือง',
  'ยะลา',
  'ทีม',
  'พื้นที่',
  'ย้ำ',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่องดื่ม',
  'ทุ่ม',
  'เคร่งครัด',
  'ลด',
  'ความเสี่ยง',
  'แพร่เชื้อ',
  'โควิด',
  'แพร่ระบาด',
  'คลัสเตอร์',
  'พื้นที่',
  'รัฐ',
  'ยะลา',
  'ร.ร.',
  'ร.ง.',
  'ยาง',
  'ยะลา',
  'เบตง',
  'ยอด',
  'ติดโรค',
  'ต่อเนื่อง'],
 ['ยะลา',
  'คุม',
  'เข้ม',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่องดื่ม',
  'ทุ่ม',
  'ลด',
  'เสี่ยง',
  'โควิด'],
 ['นอ.',
  'เมือง',
  'ยะลา',
  'ทีม',
  'พื้นที่',
  'ย้ำ',
  'มาตรการ',
  'เคอร์ฟิว',
  'ร้าน',
  'ขาย',
  'อาหาร',
  'เครื่องดื่ม',
  'ทุ่ม',
  'เคร่งครัด',
  'ลด',
  'ความเสี่ยง',
  'แพร่เชื้อ',
  'โควิด']]

**Step 3 + 4a: Bag of words + count word**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
tokens_list_j = [','.join(tkn) for tkn in tokens_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
print(cvec)
c_feat = cvec.fit_transform(tokens_list_j)

CountVectorizer(analyzer=<function <lambda> at 0x7f37aef54f80>, binary=False,
                decode_error='strict', dtype=<class 'numpy.int64'>,
                encoding='utf-8', input='content', lowercase=True, max_df=1.0,
                max_features=None, min_df=1, ngram_range=(1, 1),
                preprocessor=None, stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)


In [ ]:
cvec.vocabulary_

{'ขาย': 0,
 'คลัสเตอร์': 1,
 'ความเสี่ยง': 2,
 'คุม': 3,
 'ติดโรค': 4,
 'ต่อเนื่อง': 5,
 'ทีม': 6,
 'ทุ่ม': 7,
 'นอ.': 8,
 'พื้นที่': 9,
 'มาตรการ': 10,
 'ยอด': 11,
 'ยะลา': 12,
 'ยาง': 13,
 'ย้ำ': 14,
 'ร.ง.': 15,
 'ร.ร.': 16,
 'รัฐ': 17,
 'ร้าน': 18,
 'ลด': 19,
 'อาหาร': 20,
 'เข้ม': 21,
 'เครื่องดื่ม': 22,
 'เคร่งครัด': 23,
 'เคอร์ฟิว': 24,
 'เบตง': 25,
 'เมือง': 26,
 'เสี่ยง': 27,
 'แพร่ระบาด': 28,
 'แพร่เชื้อ': 29,
 'โควิด': 30}

In [ ]:
c_feat[:,:].todense()

matrix([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
         1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
         0, 1, 1, 1, 1, 1, 0, 1, 1, 1],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
         1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
        [1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
         0, 1, 1, 1, 0, 1, 0, 0, 1, 1]])

In [ ]:
#first_news เป็น array ของข่าว โดย row ที่ 1 = ข่าวที่ต้องการตรวจสอบจาก cofact, row ที่ 2 = ข้อความสรุปใน cofact, row ที่ 3 = หัวข้อข่าวที่เชื่อถือได้ , row ที่ 4 = เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้
first_news = c_feat[:,:].todense()
print(first_news) 

[[1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0]
 [1 1 1 0 1 1 1 1 1 2 1 1 3 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1]
 [1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 1 0 0 1]
 [1 0 1 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 1]]


In [ ]:
first_news.shape[1]

31

In [ ]:
#แปลงเป็น array ทััง 3 ข่าวเพื่อหา cosine similarity
cofact_checker_array = numpy.array(cofact_checker) 
title_news_array = numpy.array(title_news) 
first_paragraph_news_array = numpy.array(first_paragraph_news)

#หา similarity เทียบระหว่างข่าวที่ต้องการตรวจสอบบน cofact กับหัวข้อข่าวที่เชื่อถือได้
print(cofact_checker_array[0][2])
print(first_paragraph_news_array[0][4])

1
2


In [ ]:
find_cosine_similarity(first_news, 0, 1) #เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ ข้อความสรุปใน cofact

This is sum (upperpart) : 10
This is lowerpart : 900


0.011111111111111112

In [ ]:
find_cosine_similarity(first_news, 0, 2)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ หัวข้อข่าวที่เชื่อถือได้

This is sum (upperpart) : 10
This is lowerpart : 169


0.05917159763313609

In [ ]:
find_cosine_similarity(first_news, 0, 3)#เทียบ ข้อความที่ต้องการตรวจสอบบน cofact กับ เนื้อหาข่าว paragraph แรกของข่าวที่เชื่อถือได้

This is sum (upperpart) : 8
This is lowerpart : 400


0.02

------------------------------------   Develop Part --------------------------------------------

**Step 3 + 4b: Bag of words + tf-idf**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','),)
t_feat = tvec.fit_transform(tokens_list_j)

In [ ]:
t_feat[:,:5].todense()

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.28332179, 0.35935776, 0.28332179, 0.17967888, 0.35935776],
        [0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.21590833, 0.        , 0.21590833, 0.        , 0.        ]])

In [ ]:
print(len(tvec.idf_),len(tvec.vocabulary_))

29 29


In [ ]:
c_feat[:,:5].todense()

matrix([[0, 0, 0, 0, 0],
        [2, 2, 2, 1, 2],
        [0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0]])